# Polars

In [29]:
import polars as pl

### Load Data

In [30]:
DATA_DIR = "../data/dota"

In [8]:
pl_match = pl.read_csv(f"{DATA_DIR}/match.csv")
pl_cluster_regions = pl.read_csv(f"{DATA_DIR}/cluster_regions.csv")
pl_purchase_log = pl.read_csv(f"{DATA_DIR}/purchase_log.csv")
pl_item_id_names = pl.read_csv(f"{DATA_DIR}/item_ids.csv")
pl_players = pl.read_csv(f"{DATA_DIR}/players.csv")

In [25]:
# add region names to match data

match_with_region = pl_match.join(pl_cluster_regions, how="left", on="cluster").drop("cluster")

In [26]:
# add item names to item purchases

purchases_with_item_names = pl_purchase_log.join(pl_item_id_names, how="left", on="item_id").drop("item_id").groupby(["match_id", "player_slot", "item_name"]).agg(pl.col("time").list().keep_name()).groupby(["match_id", "player_slot"]).agg(
    [
        pl.apply(
            [pl.col("item_name"), pl.col("time")],
            lambda s: dict(zip(s[0], s[1].to_list())),
        ).alias("purchases")
    ]
)

purchases_with_item_names.head(15)

match_id,player_slot,purchases
i64,i64,object
20638,3,"{'heart': [1968], 'stout_shield': [-85], 'tpscroll': [227, 227, 543, 543, 909, 909, 1065, 1374, 1375, 1949, 1949], 'maelstrom': [1370], 'tango': [-85], 'reaver': [1629], 'power_treads': [871], 'blink': [699], 'branches': [-84], 'quelling_blade': [-85], 'boots': [173], 'demon_edge': [2127], 'gloves': [847, 918], 'belt_of_strength': [788], 'vitality_booster': [1708], 'javelin': [2601], 'mithril_hammer': [1301]}"
47482,131,"{'ring_of_basilius': [67], 'blades_of_attack': [296, 296], 'claymore': [867], 'staff_of_wizardry': [1523], 'sobi_mask': [67], 'tango': [-56], 'ultimate_scepter': [1537], 'talisman_of_evasion': [1847], 'dust': [346], 'tpscroll': [18, 460, 462, 465, 610, 709, 911, 1022, 1075, 1203, 1477, 1542, 1814], 'ring_of_health': [568], 'gauntlets': [457], 'ogre_axe': [1523], 'butterfly': [1854], 'quelling_blade': [-54], 'boots': [208], 'branches': [-53], 'pers': [705], 'blade_of_alacrity': [1523], 'circlet': [24, 457], 'void_stone': [705], 'wraith_band': [386], 'broadsword': [978], 'eagle': [1668], 'phase_boots': [296], 'ring_of_aquila': [386], 'stout_shield': [-60], 'ring_of_protection': [-58], 'quarterstaff': [1769], 'bracer': [457], 'point_booster': [1537], 'mithril_hammer': [2068], 'slippers': [385], 'bfury': [1012], 'boots_of_elves': [1023]}"
41525,132,"{'tpscroll': [15, 434, 708, 819, 1006, 1012, 1102, 1290, 1405, 1581], 'mithril_hammer': [2674], 'travel_boots': [2781], 'armlet': [1010], 'stout_shield': [-86], 'heart': [2176], 'blades_of_attack': [1010], 'belt_of_strength': [610], 'vitality_booster': [2025], 'ring_of_regen': [243], 'magic_wand': [427], 'platemail': [1352], 'gloves': [762, 838], 'soul_ring': [243], 'assault': [1578], 'helm_of_iron_will': [1010], 'boots': [290, 2781], 'hyperstone': [1255], 'quelling_blade': [564], 'magic_stick': [427], 'reaver': [2023], 'power_treads': [762], 'branches': [-85, -84], 'circlet': [427], 'chainmail': [1354], 'tango': [-85], 'sobi_mask': [243]}"
12644,4,"{'crimson_guard': [2803], 'buckler': [2803], 'ward_observer': [255], 'arcane_boots': [540], 'bracer': [383], 'stout_shield': [-64], 'robe': [925, 1206], 'clarity': [-55], 'belt_of_strength': [2335], 'vitality_booster': [1737], 'ward_sentry': [719, 1226, 1299, 1299, 1426, 2363, 2570], 'magic_wand': [258], 'magic_stick': [-64], 'broadsword': [754], 'branches': [-62, 248, 2803], 'chainmail': [1092, 2803], 'circlet': [248, 383], 'energy_booster': [540], 'boots': [154], 'heavens_halberd': [2335], 'gauntlets': [383], 'ogre_axe': [2335], 'blade_mail': [1216], 'ring_of_health': [1997], 'tpscroll': [239, 392, 591, 648, 785, 882, 2680], 'flask': [251], 'vanguard': [2007], 'sange': [2335], 'talisman_of_evasion': [2154], 'tango': [-63]}"
11502,4,"{'travel_boots': [3222], 'flask': [-76], 'tpscroll': [440, 459, 593, 811, 932, 943, 943, 1375, 1517, 1520, 1774, 2014, 2014, 3023, 3024], 'pipe': [2141], 'ring_of_health': [1906], 'relic': [2566], 'buckler': [1355], 'sobi_mask': [136], 'tango': [-77], 'headdress': [1355, 2083], 'blink': [885], 'branches': [-71, -71, 1126, 1299, 1300, 2083], 'chainmail': [1299], 'circlet': [1126], 'magic_stick': [441], 'boots': [236, 3222], 'energy_booster': [361], 'radiance': [3018], 'soul_ring': [181], 'cloak': [1910], 'clarity': [-71, -71], 'guardian_greaves': [1789], 'ring_of_regen': [-74, 1299, 1931, 1958, 2083], 'magic_wand': [1139], 'mekansm': [1391], 'arcane_boots': [361], 'hood_of_defiance': [1978]}"
24586,128,"{'tango': [-74], 'oblivion_staff': [739, 934], 'ultimate_scepter': [2187], 'dust': [1827, 1827, 1830], 'blades_of_attack': [-77, 120], 'orchid': [1099], 'quarterstaff': [633, 919], 'travel_boots': [2774], 'point_booster': [1365], 'phase_boots': [229], 'robe': [407, 461], 'blade_of_alacrity': [2112], 'blink': [2327], 'branches': [-76], 'boots': [211, 2774], 'tpscroll': [410, 481, 679, 790, 1076, 1283, 1284, 1596, 2188, 2435], 'ogre_axe': [1695], 'staff_of_wizardry': [1904], 'sobi_mask': [722, 920]}"
15257,128,"{'broadsword': [2416], 'blade_of

In [16]:
pl_purchase_log.head(3)

item_id,time,player_slot,match_id
i64,i64,i64,i64
44,-81,0,0
29,-63,0,0
43,6,0,0
